<a href="https://colab.research.google.com/github/tcglarry/geo/blob/master/geo_stage_2_11x11x5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""ali_build_model_stage_2_MobileNetV2_aug_1000_dense.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1wSVEmcoWzRm-7B6UGkHB8ysfMDRT_xK5
"""

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
src=  'drive/My Drive/geo/'

In [4]:

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, \
Conv2DTranspose,BatchNormalization
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import regularizers

Using TensorFlow backend.


In [5]:
data= np.load(src+'train_data.npy')
label = np.load(src+'train_labels.npy')
label = label[:,:,:,np.newaxis]
print (data.shape)
print (label.shape)



(4200, 21, 21, 20)
(4200, 20, 20, 1)


# Reduce H,W from 21 to 11, Reduce channel from 21 to 11





In [6]:
index_list_2 = list(range(20))
select_list_2 = [idx for idx in index_list_2 if idx % 4  == 0 ]

index_list_2 = list(range(21))
select_list_0 = [idx for idx in index_list_2 if idx % 2  == 0 ]
print (select_list_2)
print (select_list_0)



[0, 4, 8, 12, 16]
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]


In [7]:
data = data[:,:,:,select_list_2]

data = data[:,select_list_0,:,:]

data = data[:,:,select_list_0,:]
data.shape

(4200, 11, 11, 5)

In [0]:
data_mean = np.mean(data)
data_std = np.std(data)
label_max = np.max(label)
label_min = np.min(label)

train_data = (data - data_mean)/data_std
#label_data = (label-label_min)/(label_max - label_min)
label_data = np.log(label)

In [0]:
train_x,val_x,train_y,val_y = train_test_split(train_data,label_data,test_size=0.2, shuffle=True)

In [0]:
def conv_block(ch, activation= 'relu', padding='same',kernel_regularizer=regularizers.l2(0.01)):       
    return Conv2D(ch,(3,3),activation= activation, padding =padding )

In [0]:
#def build_model_1(IMAGE_HEIGHT=IMAGE_HEIGHT,IMAGE_WIDTH=IMAGE_WIDTH,ch=ch):
def build_model_1(ch=8):
    #inputs = Input((IMAGE_HEIGHT,IMAGE_WIDTH,ch))
    inputs = Input((11,11,5))
    
    up_1 = UpSampling2D(size=(2,2))(inputs)

    conv0 = Conv2D(32,(3,3),padding='valid')(up_1)
    print ('conv0',conv0.get_shape())
    conv1 =  conv_block(ch)(conv0)
    conv1 = conv_block(ch)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    print ('pool1',pool1.get_shape())
    
    conv2 =  conv_block(ch*2)(pool1)
    conv2 = conv_block(ch*2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    print ('poo12',pool2.get_shape())
    
    conv_test = Conv2D(64,(2,2),padding='valid')(pool2)
    print ('conv_test',conv_test.get_shape())
    
    
    
    conv3 =  conv_block(ch*4)(conv_test)
    conv3 = conv_block(ch*4)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    print ('poo13',pool3.get_shape())
    
    conv4 =  conv_block(ch*8)(pool3)
    conv4 = conv_block(ch*8)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    print ('poo14',pool4.get_shape())
    
    conv5 =  conv_block(ch*16)(pool4)
    conv5 = conv_block(ch*16)(conv5)

    
    up6 = concatenate([UpSampling2D(size=(2,2))(conv5), conv4], axis=3)
    conv6 = conv_block(ch*8)(up6)
    conv6 = conv_block(ch*8)(conv6)
    
    up7 = concatenate ([UpSampling2D(size=(2,2))(conv6), conv3],  axis=3)
    conv7 = conv_block(ch*4)(up7)
    conv7 = conv_block(ch*4)(conv7)
    print ('conv7',conv7.get_shape())
    conv7 = Conv2DTranspose(ch*4,(2,2),padding='valid')(conv7)
    print ('conv7',conv7.get_shape())
    
    up8 = concatenate([UpSampling2D(size=(2,2))(conv7), conv2], axis=3)
    conv8 = conv_block(ch*2)(up8)
    conv8 = conv_block(ch*2)(conv8)
    
    up9 = concatenate([UpSampling2D(size=(2,2))(conv8), conv1],  axis=3)
    conv9 = conv_block(ch)(up9)
    conv9 = conv_block(ch)(conv9)
    
    
    
    conv10 = Conv2D(1,(1,1))(conv9)
    
  
    model = Model(inputs= inputs, outputs=conv10)
    model.summary()
    
    return model
    
    

In [12]:
model = build_model_1()

conv0 (?, 20, 20, 32)
pool1 (?, 10, 10, 8)
poo12 (?, 5, 5, 16)
conv_test (?, 4, 4, 64)
poo13 (?, 2, 2, 32)
poo14 (?, 1, 1, 64)
conv7 (?, 4, 4, 32)
conv7 (?, ?, ?, 32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 11, 11, 5)    0                                            
__________________________________________________________________________________________________
up_sampling2d_1 (UpSampling2D)  (None, 22, 22, 5)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 20, 20, 32)   1472        up_sampling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 20

In [0]:
model.compile (optimizer='Adam', loss = 'mse', metrics =['mae'])

In [14]:

# train the model on the new data for a few epochs

use_model= False
if use_model:
    

    if os.path.isfile(src+'water_first_try.h5'):

      model = load_model(src+'water_first_try.h5')
      print ('model laoded')
    else:
      print ('model not exist')
else:
  print ('not to use model')


not to use model


In [15]:





checkpoint = ModelCheckpoint(filepath=src+'geo_stage2_2nd_try_11_11_5.h5', monitor = 'val_loss', save_best_only=True, mode= 'auto')
earlystop = EarlyStopping(patience=30)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=15, min_lr=0.00001,verbose=1)
callback_list = [checkpoint, earlystop]
model.fit(x=train_x, y=train_y, batch_size=32, epochs=1000, callbacks = callback_list, verbose=1, validation_split=0.2,  shuffle=True)

Train on 2688 samples, validate on 672 samples
Epoch 1/1000
2688/2688 [==============================] - 5s 2ms/step - loss: 1.1089 - mean_absolute_error: 0.8306 - val_loss: 0.7611 - val_mean_absolute_error: 0.6961
Epoch 2/1000
2688/2688 [==============================] - 2s 904us/step - loss: 0.6699 - mean_absolute_error: 0.6506 - val_loss: 0.5846 - val_mean_absolute_error: 0.6066
Epoch 3/1000
2688/2688 [==============================] - 2s 913us/step - loss: 0.5067 - mean_absolute_error: 0.5615 - val_loss: 0.4469 - val_mean_absolute_error: 0.5278
Epoch 4/1000
2688/2688 [==============================] - 2s 902us/step - loss: 0.4178 - mean_absolute_error: 0.5086 - val_loss: 0.3834 - val_mean_absolute_error: 0.4866
Epoch 5/1000
2688/2688 [==============================] - 2s 921us/step - loss: 0.3654 - mean_absolute_error: 0.4745 - val_loss: 0.3531 - val_mean_absolute_error: 0.4656
Epoch 6/1000
2688/2688 [==============================] - 2s 909us/step - loss: 0.3334 - mean_absolute_er

In [16]:

error = model.evaluate(val_x,val_y)
print (error)

840/840 [==============================] - 0s 272us/step
[0.17533326943715413, 0.32571662437348137]


# Testing Set NRMSE Loss

In [17]:



def nrmse(y_true,y_pred):

    return (np.sqrt(np.mean(np.square(y_true - y_pred))))/np.mean(y_pred)
def calculate_nrmse():
  y_pred = np.exp(model.predict(val_x))
  print (f'shape of y_pred {y_pred.shape}')
  y_true = np.exp(val_y)
  print (f'shape of y_true {y_true.shape}')

  loss =  nrmse(y_true,y_pred)

  print (loss)

  return loss


nrmse_loss = calculate_nrmse()
print ('claculate valuatdaion NRMSE loss', nrmse_loss)

print ('\n', 'start preparing test data')
#print (os.path.isfile(src+'test2.npy'))
test = np.load(src+'test2.npy')
test_data = (test - data_mean)/data_std

test_data = test_data[:,:,:,select_list_2]

test_data = test_data[:,select_list_0,:,:]

test_data = test_data[:,:,select_list_0,:]
print ('test_data shape', test_data.shape)


prediction = model.predict(test_data)
prediction = np.exp(prediction)
pred = prediction.reshape(1800,-1)
#print (pred.shape)



src_2 =  'drive/My Drive/geo/test_data/test_dataset/'

test_label_df = pd.read_csv(src_2+'solution.csv')



test_label_df = test_label_df.drop('ID', axis=1)


test_label = test_label_df.values


print ('test set NRMSE error', nrmse(test_label,pred))

shape of y_pred (840, 20, 20, 1)
shape of y_true (840, 20, 20, 1)
0.6407559416993558
claculate valuatdaion NRMSE loss 0.6407559416993558

 start preparing test data
test_data shape (1800, 11, 11, 5)
test set NRMSE error 0.6391625817462362
